# 1. Data collection notebook

### Objectives
Compile a library containing historical events (relavant to industry) with historical and the most recent market data for 3 leading Li mining companies.

### Inputs
* Market data from yfinance
* Library of historical events (generated by ChatGPT)

### Outputs
`output\nb1_market_index_with_historical.feather`

### Notes/comments

<hr>

### Import required libraries

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import warnings  # Filter warnings from yfinance
import matplotlib.pyplot as plt
from datetime import date, timedelta

#### Config, adjust settings

In [ ]:
warnings.simplefilter('ignore') # suppress verbosity from yfinance library

#### Get stock data

In [ ]:
tickers = ['ALB', 'SQM', 'PLL']  # We drop 'LAC' since it is relatively young company
start_date = "2010-01-01"
# end_date = "2025-04-04"
today = date.today()
# Go back to the last Sunday
days_since_sunday = (today.weekday() + 1) % 7
last_sunday = today - timedelta(days=days_since_sunday)
end_date = last_sunday.strftime("%Y-%m-%d")
# Download weekly adjusted close prices
def get_weekly_data(tickers, start, end):
    df = yf.download(tickers, start=start, end=end, interval='1wk')['Close']
    df = df.dropna(how='all')  # drop rows with all NaNs
    return df

price_df = get_weekly_data(tickers, start_date, end_date)
price_df.head()

In [ ]:
# Fetch and plot data
for ticker in tickers:
    data = yf.download(ticker, start='2010-01-01', end=end_date)
    plt.plot(data['Close'], label=ticker)

plt.title("Lithium Mining Companies Stock Prices")
plt.xlabel("Date")
plt.ylabel("Price (USD)")
plt.legend()
plt.grid(True)
plt.savefig('../assets/img/mining_comp_stock_price.png', dpi=300)
plt.show()


In [ ]:
price_df.info()

#### Calculate weekly return

In [ ]:
# Calculate weekly returns for each ticker
returns_df = price_df.pct_change().dropna()
returns_df.head()


#### Create a lithium sentiment index

In [ ]:
# Simple average across all 3 tickers. This line creates a synthetic sentiment index that rises or falls depending on the average performance of the stocks.
returns_df['Lithium_Index'] = returns_df.mean(axis=1)


#### Convert sentiment into Bullish/Bearish labels (for classification)

In [ ]:
returns_df['Sentiment_Label'] = returns_df['Lithium_Index'].apply(lambda x: 1 if x > 0 else 0)


In [ ]:
returns_df.head()

In [ ]:
returns_df['Sentiment_Label'].unique()  # Sentiment label: 1 - Bullish, 0 - Bearish

In [ ]:
returns_df['Lithium_Index'].plot(figsize=(12, 5), title='Lithium Market Sentiment Index (Avg Weekly Return)')
plt.axhline(0, color='red', linestyle='--')
plt.ylabel("Avg Weekly Return")
plt.grid(True)
plt.show()


#### Turn returns into a Lithium market index level

In [ ]:
# Start index at 100 (like many indices do)
returns_df['Lithium_Market_Index'] = (1 + returns_df['Lithium_Index']).cumprod() * 100


So now:
* The index starts at 100
* If the average weekly return is positive, the index goes up; if negative, it drops
* It mimics the feel of a real price index over time — just based on the lithium sentiment

In [ ]:
returns_df['Lithium_Market_Index'].plot(figsize=(12,5), title="Synthetic Lithium Market Sentiment Index")
plt.ylabel("Index Level")
plt.grid(True)
plt.show()


In [ ]:
returns_df.tail()

### Build a library of historical events:

There is no direct way in yfinance itself to “attach” historical events to a stock’s price data. Typically, you would pull the historical price data using yfinance and then merge or annotate this data with your own external list of events (e.g., earnings releases, product announcements, geopolitical events, etc.). You can store these events in a separate pandas DataFrame or dictionary and then join or annotate your price DataFrame accordingly.

In [ ]:
# Note: yyyy-mm-dd is the standard yfinance data format

li_ion_events = {
    "1976-01-01": "John B. Goodenough proposes cobalt oxide as a cathode material.",
    "1980-01-01": "Introduction of the layered cathode structure for lithium-ion batteries by John B. Goodenough.",
    "1985-01-01": "Akira Yoshino develops the first prototype of a lithium-ion battery.",
    "1991-01-01": "Sony commercializes the first lithium-ion rechargeable battery.",
    "2008-01-01": "Tesla launches the Roadster, the first EV using Li-ion batteries.",
    "2010-12-01": "Nissan Leaf becomes the world's first mass-produced EV with Li-ion batteries.",
    "2012-06-01": "Tesla Model S launches, popularizing electric vehicles with long-range capability.",
    "2016-07-01": "Tesla's Gigafactory 1 in Nevada begins operation.",
    "2017-01-01": "China launches significant subsidies for EVs, increasing battery production facilities.",
    "2018-01-01": "LG Chem announces major battery plant in Poland.",
    "2019-10-09": "Nobel Prize awarded to Goodenough, Whittingham, and Yoshino for their work on Li-ion batteries.",
    "2019-01-01": "Hyundai Kona Electric and Tesla Model 3 redefine affordable long-range EVs.",
    "2020-01-01": "Tesla's Gigafactory Shanghai becomes fully operational.",
    "2020-03-01": "COVID-19 pandemic disrupts global supply chains.",
    "2022-01-01": "Solid-state batteries reach early commercial prototypes.",
    "2022-08-01": "Lithium prices skyrocket due to supply-demand imbalances.",
    "2023-01-01": "U.S. Inflation Reduction Act provides subsidies for EVs and domestic battery manufacturing.",
    "2023-01-01": "Major advancements in lithium recycling technology.",
    "2023-06-01": "CATL announces new energy-dense battery for EVs.",
    "2024-01-01": "EU announces strict EV regulations, banning internal combustion engine sales by 2035.",
    "2025-01-01": "Tesla's Gigafactory Berlin expected to reach peak production capacity."
}


* Adding a simple binary `Has_Event` column based on whether there's a historic lithium event on a given day — treating multiple events on the same date as just "1"
* Aligning the event dates to the nearest valid trading day in your DataFrame. Here's a clean way to do that

In [ ]:
# Your original DataFrame
# Ensure datetime index is sorted
returns_df.index = pd.to_datetime(returns_df.index)
returns_df.sort_index(inplace=True)

# Parse event dates
raw_event_dates = pd.to_datetime(list(li_ion_events.keys()))

# Snap each event to the next available trading date (or last one if out of range)
indices = returns_df.index.searchsorted(raw_event_dates)
indices = np.minimum(indices, len(returns_df.index) - 1)  # prevent out-of-bounds

# Get actual dates from the index
valid_event_dates = returns_df.index[indices]

# Mark them in the DataFrame
returns_df["Has_Event"] = returns_df.index.isin(valid_event_dates).astype(int)

In [ ]:
returns_df.head()

In [ ]:
# Basic line plot
plt.figure(figsize=(14, 6))
plt.plot(returns_df.index, returns_df["Lithium_Market_Index"], label="Lithium Market Index", linewidth=2)

# Overlay event markers
for date in returns_df[returns_df["Has_Event"] == 1].index:
    plt.axvline(x=date, color="red", linestyle="--", alpha=0.6)

# Labels and styling
plt.title("Lithium Market Index with Historical Events")
plt.xlabel("Date")
plt.ylabel("Index Value")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig('../assets/img/li_index_with_historical_events.png', dpi=300)
plt.show()

In [ ]:
returns_df.to_feather('../output/nb1_market_index_with_historical.feather')